In [ ]:
import numpy as np
from scipy.optimize import fsolve
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.core.problem import ElementwiseProblem
from pymoo.optimize import minimize
import matplotlib.pyplot as plt


In [ ]:

#### ODEs for the posneg circuit
def Equ1(x, y, beta_x, n):
    """
    Compute the first ODE: dx/dt = beta_x / (1 + y^n) - x.

    Args:
        x (float): State variable x.
        y (float): State variable y.
        beta_x (float): Parameter controlling production rate of x.
        n (float): Parameter controlling nonlinearity.

    Returns:
        float: Value of dx/dt.
    """
    return beta_x / (1 + y**n) - x

def Equ2(x, y, beta_y, n):
    """
    Compute the second ODE: dy/dt = beta_y * x^n / (1 + x^n) - y.

    Args:
        x (float): State variable x.
        y (float): State variable y.
        beta_y (float): Parameter controlling production rate of y.
        n (float): Parameter controlling nonlinearity.

    Returns:
        float: Value of dy/dt.
    """
    return beta_y * x**n / (1 + x**n) - y

def Equs(P, t, params):
    """
    Evaluate the vector field for the ODE system.

    Args:
        P (array): State vector [x, y].
        t (float): Time (unused, as ODE is autonomous).
        params (array): Parameters [beta_x, beta_y, n].

    Returns:
        array: ODE values [dx/dt, dy/dt].
    """
    x, y = P
    beta_x, beta_y, n = params
    return np.array([Equ1(x, y, beta_x, n), Equ2(x, y, beta_y, n)])

#### Sensitivity functions
def S_betax_xss_analytic(xss, yss, beta_x, beta_y, n):
    """
    Calculate sensitivity of xss to beta_x.

    Args:
        xss (float): Steady state value of x.
        yss (float): Steady state value of y.
        beta_x (float): Parameter beta_x.
        beta_y (float): Parameter beta_y.
        n (float): Parameter n.

    Returns:
        float: Absolute value of sensitivity.
    """
    numer = beta_x * (1 + xss**n) * (1 + yss**n)
    denom = n**2 * beta_x * yss**n + xss * (1 + yss**n)**2 + xss**(1+n) * (1 + yss**n)**2
    return abs(numer / denom)

def S_betax_yss_analytic(xss, yss, beta_x, beta_y, n):
    """
    Calculate sensitivity of yss to beta_x.

    Args:
        xss (float): Steady state value of x.
        yss (float): Steady state value of y.
        beta_x (float): Parameter beta_x.
        beta_y (float): Parameter beta_y.
        n (float): Parameter n.

    Returns:
        float: Absolute value of sensitivity.
    """
    numer = n * beta_x * beta_y * xss**(n-1)
    denom = (1 + xss**n)**2 * yss + n**2 * beta_y * xss**n * yss**n + (1 + xss**n)**2 * yss**(1+n)
    return abs(numer / denom)

#### Initial guesses for steady state
def generate_initial_guesses(beta_x_val, beta_y_val):
    """
    Provide initial guesses for solving the ODE system.

    Args:
        beta_x_val (float): Value of beta_x.
        beta_y_val (float): Value of beta_y.

    Returns:
        list: List of initial guess arrays [x, y].
    """
    return [
        np.array([1, 1]),
        np.array([0, 0]),
        np.array([0.3, 12.5]),
        np.array([50.5, 0.9]),
        np.array([67.6, 0.9]),
        np.array([21, 0.9])
    ]

def ssfinder(beta_x_val, beta_y_val, n_val):
    """
    Find the steady state (xss, yss) for given beta_x, beta_y, and n.

    Args:
        beta_x_val (float): Value of beta_x.
        beta_y_val (float): Value of beta_y.
        n_val (float): Value of n.

    Returns:
        tuple: (xss, yss) if valid, else (np.nan, np.nan).
    """
    params = np.array([beta_x_val, beta_y_val, n_val])
    for init_guess in generate_initial_guesses(beta_x_val, beta_y_val):
        #### Solve ODE system using fsolve
        output, infodict, intflag, _ = fsolve(
            Equs, init_guess, args=(0, params), xtol=1e-12, full_output=True
        )
        xss, yss = output
        fvec = infodict['fvec']
        #### Check stability via Jacobian eigenvalues
        delta = 1e-8
        dEqudx = (Equs([xss + delta, yss], 0, params) - Equs([xss, yss], 0, params)) / delta
        dEqudy = (Equs([xss, yss + delta], 0, params) - Equs([xss, yss], 0, params)) / delta
        jac = np.transpose(np.vstack((dEqudx, dEqudy)))
        eig = np.linalg.eig(jac)[0]
        instability = np.any(np.real(eig) >= 0)
        #### Validate solution: positive, small residual, stable, and converged
        if (xss > 0.04 and yss > 0.04 and np.linalg.norm(fvec) < 1e-10 and
                intflag == 1 and not instability):
            return xss, yss
    return np.nan, np.nan

class Sensitivity(ElementwiseProblem):
    """
    Define the multi-objective optimization problem to minimize sensitivities.

    Optimizes beta_x, beta_y, and n to minimize S_betax_xss and S_betax_yss.
    """
    def __init__(self):
        super().__init__(
            n_var=3,  #### Variables: beta_x, beta_y, n
            n_obj=2,  #### Objectives: S_betax_xss, S_betax_yss
            n_constr=0,
            xl=np.array([0.001, 0.001, 0.001]),  #### Lower bounds
            xu=np.array([50.0, 50.0, 10.0])      #### Upper bounds
        )

    def _evaluate(self, x, out, *args, **kwargs):
        """
        Evaluate the objectives for a given (beta_x, beta_y, n) set.

        Args:
            x (array): Parameter vector [beta_x, beta_y, n].
            out (dict): Dictionary to store objective values.
        """
        beta_x, beta_y, n = x
        xss, yss = ssfinder(beta_x, beta_y, n)
        if np.isnan(xss) or np.isnan(yss):
            out["F"] = [1e5, 1e5]  #### Penalize invalid solutions
        else:
            out["F"] = [
                S_betax_xss_analytic(xss, yss, beta_x, beta_y, n),
                S_betax_yss_analytic(xss, yss, beta_x, beta_y, n)
            ]


In [ ]:

algorithm = NSGA2(pop_size=1000)  #### Population size
res = minimize(
    Sensitivity(),
    algorithm,
    ('n_gen', 200),  #### Number of generations
    seed=1,
    verbose=True
)


In [ ]:

F = res.F  #### Objective values (sensitivities)
X = res.X  #### Parameter values (beta_x, beta_y, n)
print(f"Found {F.shape[0]} non-dominated points in {res.exec_time:.3f}s")

plt.figure(figsize=(8, 6))
plt.scatter(F[:, 0], F[:, 1], s=10)
plt.xlabel(r'$S_{\beta_x,x_{ss}}$')  #### Sensitivity of xss to beta_x
plt.ylabel(r'$S_{\beta_x,y_{ss}}$')  #### Sensitivity of yss to beta_x
plt.title('NSGA-II Pareto Front for posneg Circuit')
plt.tight_layout()
plt.show()